In [1]:
test_source_path = 'D:/Summarization/datasets/cnn_dm/corrupted_nodup_files/test.source'
test_target_path = 'D:/Summarization/datasets/cnn_dm/corrupted_nodup_files/test.target'
pred_file_path = 'nodup_preds.hypo'

In [2]:
def read_summaries(file_path):
    lines = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            lines.append(line.strip())
    return lines

In [3]:
sources = read_summaries(test_source_path)
targets = read_summaries(test_target_path)
print(len(sources))
assert len(sources) == len(targets)

11490


In [4]:
predicts = read_summaries(pred_file_path)

In [5]:
print(len(predicts))

11490


In [6]:
index = len(sources) // 2
print('- Corrupted:')
print(sources[index][: sources[index].find('</s>')])
print('- Target:')
print(targets[index])
print('- Prediction:')
print(predicts[index])

- Corrupted:
Gertrude Weaver became the world's oldest person last week following the death of a 117-year-old woman in Japan . Waver died from complications due to pneumonia in Camden . She attributed his long life to treating others well and eating her own cooking . Weaver was born in Arkansas in 1898 and worked as a domestic helper . 115-year-old Jeralean Talley, of Detroit, is now the world's oldest person . 
- Target:
Gertrude Weaver became the world's oldest person last week following the death of a 117-year-old woman in Japan . Waver died from complications due to pneumonia in Camden . She attributed her long life to treating others well and eating her own cooking . Weaver was born in Arkansas in 1898 and worked as a domestic helper . 115-year-old Jeralean Talley, of Detroit, is now the world's oldest person .
- Prediction:
Gertrude Weaver became the world's oldest person last week following the death of a 117-year-old woman in Japan . Waver died from complications due to pneumon

In [7]:
match_num, print_count = 0, 0
clean_total, clean_match = 0, 0
clean_flag = False

for i, (s, t, p) in enumerate(zip(sources, targets, predicts)):    
    if p[0] == p[1] or p[0] == '"' or p[0] == "'":
        p = p[1:]
    if len(p) > len(t):
        p = p[: len(t)]

    s = s[: s.find('</s>') -1 ]
    t_tokens = t.split()
    p_tokens = p.split()

    if s == t:
        clean_total += 1
        clean_flag = True
    
    if t_tokens == p_tokens:
        match_num += 1
        if clean_flag:
            clean_match += 1
            clean_flag = False
    else:
        if print_count < 10 and s != t:
            print('Target:')
            print(t)
            print('Sources:')
            print(s)
            print('Prediction:')
            print(p)
            print()
            print_count += 1

Target:
Student is no longer on Duke University campus and will face disciplinary review . School officials identified student during investigation and the person admitted to hanging the noose, Duke says . The noose, made of rope, was discovered on campus about 2 a.m.
Sources:
Student is no longer on Duke University campus and will face disciplinary review . School officials identified student during investigation and the person admitted to hanging the noose, Duke says . The noose, made of rope, was discovered on campus about afternoon
Prediction:
Student is no longer on Duke University campus and will face disciplinary review . School officials identified student during investigation and the person admitted to hanging the noose, Duke says . The noosed, made of rope, was discovered on campus about 2 a.m

Target:
Former GOP representative compares President Obama to Andreas Lubitz . Bachmann said with possible Iran deal, Obama will fly "entire nation into the rocks" Reaction on social m

In [8]:
match_num / len(targets)

0.6021758050478677

In [9]:
clean_match / clean_total

0.8428198433420365

In [10]:
(match_num - clean_match) / (len(targets) - clean_total)

0.36153176675369886